In [1]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine

In [3]:

steam_df = pd.read_csv('data/steam.csv')
desc_df = pd.read_csv('data/steam_description_data.csv')
media_df = pd.read_csv('data/steam_media_data.csv')

## Working on Data to sql 

In [4]:
game_data = pd.merge(steam_df,media_df,right_on='steam_appid',left_on='appid')[['appid','name','header_image']]

In [5]:
engine = create_engine('sqlite:///game_data.db')

In [6]:
game_data.to_sql('games', con=engine, if_exists='replace', index=False)

27075

## Game Recommendation system

In [51]:
df = steam_df.copy()
df = df[['appid','name','genres','categories','steamspy_tags',]]
df1 =desc_df.copy()

In [52]:
df = df.merge(df1,left_on='appid', right_on='steam_appid')

In [53]:
df['about'] = df['categories']+ df['genres']+ df['steamspy_tags']+ df['detailed_description']
df = df[['appid','name','about']]

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000, stop_words='english')
vector=cv.fit_transform(df['about'].values.astype('U')).toarray()
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [47]:
import pickle

In [56]:
pickle.dump(df, open('game_data.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))